# Parquet Taxi Data

## Импорт библиотек

#### Импорт Spark dependencies для подключения Spark

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`;

import $ivy.$                                  ;

In [ ]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._

In [ ]:
import org.apache.log4j.{Level, Logger};
Logger.getLogger("org").setLevel(Level.OFF);

import org.apache.log4j.{Level, Logger};


## Создание спарк-сессии

In [ ]:
import org.apache.spark.sql._

val spark = SparkSession.
            builder().
            appName("Taxi_parquet").
            master("spark://spark-master:7077").
            config("spark.executor.memory", "512m").
            getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@56dc7c45

## Создается Dataframe (читаем csv формат)

### Путь до файла

In [ ]:
val path = "data/yellow_tripdata_2020-01.csv"

path: String = "data/yellow_tripdata_2020-01.csv"

### Схема для DataFrame

In [ ]:
val schema = StructType(Array(
      StructField("VendorID", IntegerType, true),
      StructField("tpep_pickup_datetime", TimestampType, true),
      StructField("tpep_dropoff_datetime", TimestampType, true),
      StructField("passenger_count", IntegerType, true),
      StructField("trip_distance", FloatType, true),
      StructField("RatecodeID", IntegerType, true),
      StructField("store_and_fwd_flag", StringType, true),
      StructField("PULocationID", IntegerType, true),
      StructField("DOLocationID", IntegerType, true),
      StructField("payment_type", IntegerType, true),
      StructField("fare_amount", FloatType, true),
      StructField("extra", FloatType, true),
      StructField("mta_tax", FloatType, true),
      StructField("tip_amount", FloatType, true),
      StructField("tolls_amount", FloatType, true),
      StructField("improvement_surcharge", FloatType, true),
      StructField("total_amount", FloatType, true),
      StructField("congestion_surcharge", FloatType, true)))

schema: StructType = List(
  StructField("VendorID", IntegerType, true, {}),
  StructField("tpep_pickup_datetime", TimestampType, true, {}),
  StructField("tpep_dropoff_datetime", TimestampType, true, {}),
  StructField("passenger_count", IntegerType, true, {}),
  StructField("trip_distance", FloatType, true, {}),
  StructField("RatecodeID", IntegerType, true, {}),
  StructField("store_and_fwd_flag", StringType, true, {}),
  StructField("PULocationID", IntegerType, true, {}),
  StructField("DOLocationID", IntegerType, true, {}),
  StructField("payment_type", IntegerType, true, {}),
  StructField("fare_amount", FloatType, true, {}),
  StructField("extra", FloatType, true, {}),
  StructField("mta_tax", FloatType, true, {}),
  StructField("tip_amount", FloatType, true, {}),
  StructField("tolls_amount", FloatType, true, {}),
  StructField("improvement_surcharge", FloatType, true, {}),
  StructField("total_amount", FloatType, true, {}),
  StructField("congestion_surcharge", FloatType, true

In [ ]:
val Taxi_df = spark.read.format("csv").schema(schema).option("header", "true").load(path)

Taxi_df: DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp ... 16 more fields]

### Вывод схемы DataFrame

In [ ]:
Taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)



### Вывод первых 2 строк необработанного DataFrame

In [ ]:
Taxi_df.show(2, truncate = 50, vertical = true)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2020-01-01 00:28:15 
 tpep_dropoff_datetime | 2020-01-01 00:33:03 
 passenger_count       | 1                   
 trip_distance         | 1.2                 
 RatecodeID            | 1                   
 store_and_fwd_flag    | N                   
 PULocationID          | 238                 
 DOLocationID          | 239                 
 payment_type          | 1                   
 fare_amount           | 6.0                 
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 1.47                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 11.27               
 congestion_surcharge  | 2.5                 
-RECORD 1------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2020-01-0

## Фильтрация данных

### Фильтрация состоит из:
* Удаление null-строк
* Добавление колонки id для каждой записи (Как id serial в SQL)

In [ ]:
val filtered_df = Taxi_df.na.drop.withColumn("id", monotonically_increasing_id())

filtered_df: DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp ... 17 more fields]

### Вывод первых 2 строк итогового DataFrame

In [ ]:
filtered_df.show(2, truncate = 50, vertical = true)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2020-01-01 00:28:15 
 tpep_dropoff_datetime | 2020-01-01 00:33:03 
 passenger_count       | 1                   
 trip_distance         | 1.2                 
 RatecodeID            | 1                   
 store_and_fwd_flag    | N                   
 PULocationID          | 238                 
 DOLocationID          | 239                 
 payment_type          | 1                   
 fare_amount           | 6.0                 
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 1.47                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 11.27               
 congestion_surcharge  | 2.5                 
 id                    | 0                   
-RECORD 1------------------------------------
 VendorID              | 1        

## Создание временного вида, чтобы можно было использовать SQL запрос

In [ ]:
filtered_df.createOrReplaceTempView("data")

## Создание итогового parquet

### Создание parquet для каждой группы пассажиров (5 Групп -  0: 0 пассажиров; 1: 1 пассажир; 2: 2 пассажира; 3: 3 пассажира; 4: 4 и более пассажиров)

In [ ]:
// Определение метода для построения parquet для группы
def get_group_parquet(sparkSession: SparkSession, sign:String, group_pas: String):DataFrame = {
    val group_parquet = sparkSession.sql(
      s"""
        |WITH pas_$group_pas AS
        |(
        |	SELECT *
        |	FROM data
        |	WHERE passenger_count $sign $group_pas
        |)
        |,
        |	parquet$group_pas AS
        |(
        |	SELECT DATE(data.tpep_pickup_datetime) as date,
        |		   (COUNT(pas_$group_pas.*) / COUNT(*) * 100.0) AS percentage_${group_pas}p,
        |		   CASE WHEN MAX(pas_$group_pas.total_amount) IS NULL THEN 0 ELSE MAX(pas_$group_pas.total_amount) END max_trip_${group_pas}p,
        |		   CASE WHEN MIN(pas_$group_pas.total_amount) IS NULL THEN 0 ELSE MIN(pas_$group_pas.total_amount) END min_trip_${group_pas}p
        |	FROM data
        |	LEFT JOIN pas_$group_pas ON data.id = pas_$group_pas.id
        |	GROUP BY date
        |)
        |SELECT *
        |FROM parquet$group_pas
        |""".stripMargin)
    return group_parquet
}

defined function get_group_parquet

### Parquet_0

In [ ]:
val parquet_0 = get_group_parquet(spark, "=", "0")

parquet_0: DataFrame = [date: date, percentage_0p: double ... 2 more fields]

### Parquet_1

In [ ]:
val parquet_1 = get_group_parquet(spark, "=", "1")

parquet_1: DataFrame = [date: date, percentage_1p: double ... 2 more fields]

### Parquet_2

In [ ]:
val parquet_2 = get_group_parquet(spark, "=", "2")

parquet_2: DataFrame = [date: date, percentage_2p: double ... 2 more fields]

### Parquet_3

In [ ]:
val parquet_3 = get_group_parquet(spark, "=", "3")

parquet_3: DataFrame = [date: date, percentage_3p: double ... 2 more fields]

### Parquet_4

In [ ]:
val parquet_4 = get_group_parquet(spark, ">=", "4")

parquet_4: DataFrame = [date: date, percentage_4p: double ... 2 more fields]

### Full_parquet

In [ ]:
val full_parquet = parquet_0
  .join(parquet_1, "date")
  .join(parquet_2, "date")
  .join(parquet_3, "date")
  .join(parquet_4, "date")
  .orderBy("date")

full_parquet: Dataset[Row] = [date: date, percentage_0p: double ... 14 more fields]

### Вывод итогового parquet

In [ ]:
// Кол-во строк в итоговом parquet
println(s"В итоговом parquet ${full_parquet.count()} строк")

В итоговом parquet 51 строк


### Для удобного отображения parquet выводим 51 строк в режиме vertical (rows & columns vertically)

In [ ]:
full_parquet.show(51, truncate = 50, vertical = true)

-RECORD 0---------------------------
 date          | 2003-01-01         
 percentage_0p | 0.0                
 max_trip_0p   | 0.0                
 min_trip_0p   | 0.0                
 percentage_1p | 100.0              
 max_trip_1p   | 0.0                
 min_trip_1p   | 0.0                
 percentage_2p | 0.0                
 max_trip_2p   | 0.0                
 min_trip_2p   | 0.0                
 percentage_3p | 0.0                
 max_trip_3p   | 0.0                
 min_trip_3p   | 0.0                
 percentage_4p | 0.0                
 max_trip_4p   | 0.0                
 min_trip_4p   | 0.0                
-RECORD 1---------------------------
 date          | 2008-12-31         
 percentage_0p | 0.0                
 max_trip_0p   | 0.0                
 min_trip_0p   | 0.0                
 percentage_1p | 80.0               
 max_trip_1p   | 73.7               
 min_trip_1p   | 7.3                
 percentage_2p | 0.0                
 max_trip_2p   | 0.0                
 

## Сохранение Итогового parquet в формате parquet и csv

### CSV формат

In [ ]:
full_parquet.repartition(1).write.format("csv").mode("overwrite").option("sep", ",").option("header", "true").save("results/parquet.csv")

### Parquet формат

In [ ]:
full_parquet.repartition(1).write.format("parquet").mode("overwrite").option("sep", ",").option("header", "true").save("results/result.parquet")